In [ ]:
################################################################################################################

# EXTERNAL MODULES TO BE USED

################################################################################################################

import pandas as pd
import secrets
import re
from PIL import Image
from flask import Flask, render_template, redirect, url_for, session, flash, request
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from flask_wtf.file import FileField, FileAllowed
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.widgets import TextArea
from wtforms.validators import InputRequired, Email, Length, ValidationError
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from datetime import datetime # timedelta





app = Flask(__name__)
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SECRET_KEY'] = 'c1155c6a351e49eba15c00ce577b259e'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

################################################################################################################

# WEB ROUTES FOR CONTROLLING ACCESS TO TEMPLATE VIEWS

################################################################################################################

@app.route("/")
def index():
    return render_template('index.html')

@app.route("/festivals")
def festivals():
    sqlalchemyObj = db.engine.execute('select * from venues ORDER BY Name')
    venues = []
    for i in sqlalchemyObj:
        venues.append(i)
    # print(venues)
    dataset2 = []
    dict={}
    for i in venues:
        dict['Name'] = i[0]
        dict['Lat'] = i[1]
        dict['Lng'] = i[2]
        dict['Url'] = i[3]
        dict['Img_url'] = i[4]
        dict['Desc'] = i[5]
        # print(i)
        # print(dict)
        dataset2.append(dict.copy()) #markers.append(fld.copy())
    print(dataset2)
    return render_template('festivals.html', entries = dataset2)

@app.route("/dressing")
def dressing():
    return render_template('dressing.html')

@app.route("/contact")
def contact():
    return render_template('contact.html')


################################################################################################################

# WEB ROUTES FOR FESTIVALS

################################################################################################################

@app.route("/1")
def festvial1():
    return render_template('1.html')

@app.route("/2")
def festvial2():
    return render_template('2.html')


################################################################################################################

# WEB ROUTES FOR ARTISTS

################################################################################################################

@app.route("/xinkuzi")
def xinkuzi():
    return render_template('xinkuzi.html')

@app.route("/mozaiyang")
@login_required
def mozaiyang():
    return render_template('mozaiyang.html')

@app.route("/wutiaoren")
@login_required
def wutiaoren():
    return render_template('wutiaoren.html')

@app.route("/masaike")
@login_required
def masaike():
    return render_template('masaike.html')


################################################################################################################

# ERROR HANDLERS

################################################################################################################

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404

################################################################################################################

# APPLICATION TEST RUN AT PORT 9002

################################################################################################################

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))


class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
    bio = db.Column(db.Text)
    image_file = db.Column(db.String(20), nullable=False, default='default.jpg')

class RegisterForm(FlaskForm):
    email = StringField("Email", validators=[InputRequired(), Email(message="Invalid Email"), Length(max=50)], render_kw={"placeholder": "邮箱：example@gmail.com"})
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "用户名：至少4个字符，至多15个字符"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "密码：至少4个字符，至多15个字符"})
    submit = SubmitField("注册")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("用户名已存在，请使用其他用户名。")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("邮箱已被注册，请使用其他邮箱。")


class LoginForm(FlaskForm):
    username = StringField("Username", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "请输入您的用户名"})
    password = PasswordField("Password", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "请输入您的密码"})
    submit = SubmitField("登录")
    
class ProfileForm(FlaskForm):
    username = StringField('Username', validators=[InputRequired(), Length(min=2, max=20)])
    email = StringField("Email", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder": "邮箱：example@gmail.com"})
    profile_body = StringField('Text', widget=TextArea())
    picture = FileField('Update Profile Picture', validators=[FileAllowed(['jpg', 'png'])])
    submit = SubmitField('Update')

class Venues(db.Model):
    __tablename__ = 'venues'
    name = db.Column(db.String(25), primary_key=True)
    Lat = db.Column(db.Float)
    Lng = db.Column(db.Float)
    url = db.Column(db.Text)
    img_url = db.Column(db.Text)
    desc = db.Column(db.Text)
    
    
@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for("index"))
        flash("用户不存在或密码错误")
    return render_template('login.html', title="Login", form=form)


@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', title='Register', form=form)


@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('index'))

@app.route("/profile", methods=['GET', 'POST'])
@login_required
def profile():
    form = ProfileForm()
    if form.validate_on_submit():
        if form.picture.data:
            picture_file = save_picture(form.picture.data)
            current_user.image_file = picture_file
        current_user.username = form.username.data
        current_user.email = form.email.data
        current_user.bio = form.profile_body.data
        db.session.commit()
        flash('Your profile has been updated!', 'success')
        return redirect(url_for('profile'))
    elif request.method == 'GET':
        form.username.data = current_user.username
        form.email.data = current_user.email
        if current_user.bio is None:
            current_user.bio = '保持神秘'
        else:
            form.profile_body.data = remove_html_tags(current_user.bio)
        if current_user.image_file is None:
            current_user.image_file = 'default.png'
    image_file = url_for('static', filename='profile_pics/' + current_user.image_file)
    return render_template('profile.html', title='Profile', image_file=image_file, form=form)

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


def save_picture(form_picture):
    random_hex = secrets.token_hex(8)
    _, f_ext = os.path.splitext(form_picture.filename)
    picture_fn = random_hex + f_ext
    picture_path = os.path.join(app.root_path, 'static/profile_pics', picture_fn)

    output_size = (100, 100)
    i = Image.open(form_picture)
    i.thumbnail(output_size)
    i.save(picture_path)

    return picture_fn




if __name__ == '__main__':
    app.run('localhost', 8008)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8008/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Dec/2021 19:51:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 19:51:13] "GET /styles/bootstrap-4.1.2/bootstrap.min.js HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 19:51:13] "GET /styles/bootstrap-4.1.2/popper.js HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 19:51:13] "GET /static/fonts/FuturaLT-Book.ttf HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 19:51:13] "GET /styles/bootstrap-4.1.2/bootstrap.min.js HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 19:51:13] "GET /static/assets/img/home_bg.png HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 19:51:13] "GET /profile HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 19:51:14] "GET /static/fonts/FuturaLT-Book.ttf HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 19:51:18] "GET /static/styles/bootstrap-4.1.2/popper.js.map HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 19:51:18] "GET /static/styles/bootstrap-4.1.2/bootstrap.min.js.map HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 19:51:18] "GET

[{'Name': '【上海】2022鸡皮疙瘩音乐节', 'Lat': 31.270009, 'Lng': 121.505284, 'Url': '#', 'Img_url': 'https://img.dahepiao.com/uploads/190920/190920/141763-1Z92009205EI.jpg', 'Desc': '[上海]上海 MODERNSKY LAB'}, {'Name': '【南京】2021南京森林音乐节江北音乐季', 'Lat': 32.084071, 'Lng': 118.709111, 'Url': '#', 'Img_url': 'https://p4.itc.cn/q_70/images03/20210622/eccfcf23fb8340cca7840f25ba274db7.png', 'Desc': '[南京]江北新区扬子江公园'}, {'Name': '【广州】2021国潮音乐嘉年华·广州站', 'Lat': 23.430868, 'Lng': 113.233549, 'Url': '#', 'Img_url': 'https://pic.rmb.bdstatic.com/bjh/news/4b37345678dfac64784df949d0d4a0f7.jpeg', 'Desc': '[广州]广州融创文旅城(融创乐园内东北侧)\xa0'}, {'Name': '【广州】2021广州草莓音乐节', 'Lat': 23.0014, 'Lng': 113.323496, 'Url': '#', 'Img_url': 'https://gimg2.baidu.com/image_search/src=http%3A%2F%2Fqqpublic.qpic.cn%2Fqq_public%2F0%2F0-3191874207-74F8F3556E0C19D94819AEFEC3727C06%2F0%3Ffmt%3Djpg%26size%3D103%26h%3D675%26w%3D900%26ppv%3D1.jpg&refer=http%3A%2F%2Fqqpublic.qpic.cn&app=2002&size=f9999,10000&q=a80&n=0&g=0n&fmt=jpeg?sec=1639927321&t=844d9d4

127.0.0.1 - - [10/Dec/2021 20:25:09] "GET /static/fonts/FuturaLT-Book.ttf HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 20:25:11] "GET /dressing HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 20:25:11] "GET /static/fonts/FuturaLT-Book.ttf HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 20:25:14] "GET /contact HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 20:25:14] "GET /static/fonts/FuturaLT-Book.ttf HTTP/1.1" 404 -
127.0.0.1 - - [10/Dec/2021 20:25:14] "GET /static/assets/images/contact.jpg HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2021 20:25:17] "GET / HTTP/1.1" 200 -
